In [ ]:
import numpy as np
np.__version__

'1.26.4'

### Load environment
Save steam_data.csv under "My Drive/data" folder

In [ ]:
# Access Google Drive
from google.colab import drive
drive.mount("/content/drive")

IN_PATH = "drive/My Drive/data/steam_data.csv"
OUT_PATH = "drive/My Drive/data/steam_clean_no_header.csv"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tqdm import tqdm
import pandas as pd
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


### Clean data

In [ ]:
tqdm.pandas()  # Progress bar

# Read csv + Drop NA Title
# Clean price: Replace "Free" with "0" + Remove "$" and "," + Convert to float
# Clean date: coerce: If a date is not in "Jan 1, 2000" format, set it to NaT/NaN
# Combine Title, Description, Tags, and Features to one column "search_text"
df = (
    pd.read_csv(IN_PATH)
    .dropna(subset=["Title"])
    .assign(
        original_price=lambda df_: df_["Original Price"]
            .replace("Free", "0")
            .str.replace(r"[$,]", "", regex=True)
            .astype(float),
        release_date=lambda df_: pd.to_datetime(
            df_["Release Date"], format="%d %b, %Y", errors="coerce"
        ),
        search_text=lambda df_: df_["Title"]
            + " " + df_["Game Description"].fillna("")
            + " " + df_["Popular Tags"].fillna("")
            + " " + df_["Game Features"].fillna(""),
    )
)

### Sentence transformer

In [ ]:
# Initialize Sentence Transformer model
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
# Other more computationally complex models
# model = SentenceTransformer("distiluse-base-multilingual-cased-v1")
# model = SentenceTransformer("xlm-r-bert-base-nli-stsb-mean-tokens")
# model = SentenceTransformer("LaBSE")

# Add embedding column with progress bar
df["embedding"] = df["search_text"].progress_apply(
    lambda text: model.encode(text).tolist()
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
  0%|          | 270/71699 [00:50<3:43:41,  5.32it/s]


KeyboardInterrupt: 

### Select columns + Write csv

In [ ]:
df[
    [
        "Title",
        "original_price",
        "release_date",
        "Game Description",
        "All Reviews Summary",
        "Developer",
        "Supported Languages",
        "Popular Tags",
        "Game Features",
        "embedding"
    ]
].to_csv(OUT_PATH, header=False, index=False)